# German Company Data Transforms - Open Registry

![opencorporate](https://opencorporates.com/assets/home/logo-97c819499e244a5ab0aacf36cc00c4f9.svg)

German Company records are avalaible in the open and in this tutorial we will be using that data. You have a financial investigation that requires to find the information about companies. The Data is released by a company called Open Corporates that is pushing for Open Data. 

# Financial Investigation Use Case

This data can help anyone looking to investigate different cases, e.g. Fraud, Who owns what, who are directors in a company?

# [Link to the Open Registry Data API](https://db.offeneregister.de/)


# Links to the Open Registry Data Download Page

https://daten.offeneregister.de/openregister.db.gz.torrent ~ 740MB
https://daten.offeneregister.de/openregister.db.gz  ~ 740 MB


Search for company_number = "H1101_H1101_HRB18423"  in DB

# [Link to the Open Corporates Data Entry](https://opencorporates.com/companies/de/H1101_H1101_HRB18423)

# Transform 1 - Search German Company

This Transform Searchs the DB for possible matches of the company

Create file

*pyconafrica/transforms/SearchCompany.py*

add code below


In [9]:
from maltego_trx.maltego import UIM_TYPES

from maltego_trx.transform import DiscoverableTransform
import sqlite3
import re


class SearchCompany(DiscoverableTransform):
    """
    This will return all the companies which contains the phrase
    
    Input Entity: Phrase
    Output Entity: Company
    """

    @classmethod
    def create_entities(cls, request, response):
        try:
            companyName = request.Value  # Value Retrieved from the Maltego Request
            companyName = re.sub('[,!.&"()-:]', ' ', companyName)  #replacing special charactes with space

            
            QUERY = "SELECT ROWID FROM company_fts WHERE name MATCH '" + companyName + "' order by rank" # uses SQLITE FTS5
            #to quickly search the DB

            conn = sqlite3.connect('C:\\handelsregister.db')# change this to path of your DB
            cursor = conn.execute(QUERY) 
            
            for row in cursor:  
                QUERY = "SELECT name, company_number, current_status, registered_address, native_company_number " \
                        "FROM company " \
                        "WHERE ROWID = " + str(row[0])
                cursor = conn.execute(QUERY)
                for row1 in cursor:
                    companyTableE = response.addEntity("maltego.Company", row1[0])
                    companyTableE.addProperty('companynumber', 'Company Number', 'strict', row1[1])
                    companyTableE.addProperty('registrationstatus', 'Registration Status', 'strict', row1[2])
                    companyTableE.addProperty('registeredaddress', 'Registered Address', 'strict', row1[3])
                    companyTableE.addProperty('nativecompanynumber', 'Native Company Number', 'strict', row1[4])
            conn.close()
        except Exception as e:
            response.addUIMessage("Error: " + str(e), UIM_TYPES["partial"])


# Transform Number 2 - Search Officers

Create file

*pyconafrica/transforms/SearchOfficers.py*

add code below

In [10]:
from maltego_trx.maltego import UIM_TYPES
from maltego_trx.entities import Person
from maltego_trx.transform import DiscoverableTransform
import sqlite3
import re



class SearchOfficers(DiscoverableTransform):
    """
    This will return all the officers within a given Company
    Input Entity: maltego.Company
    Output Entity: maltego.Person

    """

    @classmethod
    def create_entities(cls, request, response):
        try:
            
            company_number = request.getProperty('companynumber')

            QUERY = " SELECT company.name, officer.position, officer.type, officer.name FROM company \
                     INNER JOIN officer ON company.company_number = officer.company_id \
                     WHERE  company.company_number = '" + company_number+ "'"

            conn = sqlite3.connect('C:\\handelsregister.db')
            rowcursor = conn.execute(QUERY)

            for row1 in rowcursor:
                officerPerson = response.addEntity(Person, row1[3])
                officerPerson.addProperty('position', 'Position', 'strict', row1[1])

            conn.close()
        except Exception as e:
            response.addUIMessage("Company Number or Company Name not found in the entity\n"    
                                  "Error: " + str(e), UIM_TYPES["partial"])
